In [1]:
datasets = [
    "contact-primary-school",
    "contact-high-school",
    "hospital-lyon",
    "email-enron",
    "email-eu",
    "ndc-substances",
    "diseasome",
    "congress-bills",
    "tags-ask-ubuntu",
]

#disgenenet is not included

In [4]:
# epidemic threshold for an SIR model
import xgi
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import eigs

def compute_DB_matrix(H):
    """
    Construct the DB matrix for a hypergraph H.
    Each hyperedge contributes weight 1/(|e|-1) between all its node pairs.
    """
    nodes = list(H.nodes)
    node_index = {node: idx for idx, node in enumerate(nodes)}
    n = len(nodes)

    # Initialize DB as sparse matrix
    DB = lil_matrix((n, n))

    # Loop through all hyperedges
    for e in H.edges.members():
        size = len(e)
        if size <= 1:
            continue # Skip single-node hyperedges

        # Weight for each node pair in this hyperedge
        weight = 1 / (size - 1)
        # Add weight to the matrix index
        for i in e:
            for j in e:
                if i != j:
                    DB[node_index[i], node_index[j]] += weight
    return DB, nodes

def compute_sir_threshold(DB, mu=0.05):
    """
    Compute critical infection rate for SIR: beta_crit = mu / lambda_max(DB)
    """
    # largest eigenvalue
    lambda_max = eigs(DB, k=1, which='LR', return_eigenvectors=False)[0].real
    # critical infection rate
    beta_crit = mu / lambda_max if lambda_max > 0 else np.inf
    return lambda_max, beta_crit

def summarize_hypergraph(H):
    """Print quick summary of structure"""
    degrees = [H.degree(node) for node in H.nodes]
    edge_sizes = [len(e) for e in H.edges.members()]
    z = np.mean(degrees)
    print(f"Mean hyperdegree (z): {z:.3f}")
    print(f"Average hyperedge size: {np.mean(edge_sizes):.3f}")
    print(f"Num nodes: {H.num_nodes}, Num edges: {H.num_edges}")

if __name__ == "__main__":
    # Load a sample hypergraph
    H = xgi.load_xgi_data("hospital-lyon", max_order=None)

    summarize_hypergraph(H)
    DB, nodes = compute_DB_matrix(H)

    # Assume recovery rate mu
    mu = 0.05
    lambda_max, beta_crit = compute_sir_threshold(DB, mu=mu)
    print(f"λ_max(DB): {lambda_max:.5f}")
    print(f"Critical infection rate beta_crit ≈ {beta_crit:.5f} for mu = {mu}")


Mean hyperdegree (z): 774.680
Average hyperedge size: 2.087
Num nodes: 75, Num edges: 27834
λ_max(DB): 1823.05156
Critical infection rate beta_crit ≈ 0.00003 for mu = 0.05


In [5]:
for data in datasets:
    print(data)
    H = xgi.load_xgi_data(data, max_order=None)

    summarize_hypergraph(H)
    DB, nodes = compute_DB_matrix(H)

    # Assume recovery rate mu
    mu = 0.05
    lambda_max, beta_crit = compute_sir_threshold(DB, mu=mu)
    print(f"λ_max(DB): {lambda_max:.5f}")
    print(f"Critical infection rate beta_crit ≈ {beta_crit:.5f} for mu = {mu}\n")
    

contact-primary-school
Mean hyperdegree (z): 925.612
Average hyperedge size: 2.096
Num nodes: 242, Num edges: 106879
λ_max(DB): 1412.99115
Critical infection rate beta_crit ≈ 0.00004 for mu = 0.05

contact-high-school
Mean hyperdegree (z): 1078.648
Average hyperedge size: 2.050
Num nodes: 327, Num edges: 172035
λ_max(DB): 3041.02918
Critical infection rate beta_crit ≈ 0.00002 for mu = 0.05

hospital-lyon
Mean hyperdegree (z): 774.680
Average hyperedge size: 2.087
Num nodes: 75, Num edges: 27834
λ_max(DB): 1823.05156
Critical infection rate beta_crit ≈ 0.00003 for mu = 0.05

email-enron
Mean hyperdegree (z): 181.851
Average hyperedge size: 2.473
Num nodes: 148, Num edges: 10885
λ_max(DB): 779.25918
Critical infection rate beta_crit ≈ 0.00006 for mu = 0.05

email-eu
Mean hyperdegree (z): 559.800
Average hyperedge size: 2.391
Num nodes: 1005, Num edges: 235263
λ_max(DB): 4877.58277
Critical infection rate beta_crit ≈ 0.00001 for mu = 0.05

ndc-substances
Mean hyperdegree (z): 40.946
Avera

In [6]:
import sys
import numpy as np
from scipy.sparse.linalg import eigs
import xgi
import matplotlib.pyplot as plt
# import hypercontagion as hc
from collections import Counter
from termcolor import colored
import Hypergraph_Models as hm
from trie import Trie
from utilities import count_subfaces, max_number_of_subfaces, missing_subfaces
from math import comb
from itertools import combinations as combs

def generate_hypergraph(edges_of_size, degree_seq, q):
    #edges = hm.chung_lu(nodes, edges_of_size, degree_seq, multisets=True)
    edges = hm.simplicial_chung_lu(degree_seq, edges_of_size, q, multisets=True)
    H = xgi.Hypergraph(edges)
    return H

def edge_size_distribution(H):
    sizes = [len(H.edges.members(e)) for e in H.edges]
    counts = Counter(sizes)
    max_size = max(counts)
    distribution = [counts.get(i, 0) for i in range(max_size + 1)]
    return distribution

In [10]:
def compute_Bcrit_for_q(num_nodes, edges, degree_seq, gamma, colors, num_graphs, ax=None):
    """
    Compute and plot critical infection rates B_crit as a function of q (simplicial ratio).
    """
    q_values = np.linspace(0, 1, 10)
    beta_crits = []

    for q in q_values:
        # generate simplicial hypergraph
        H = generate_hypergraph(edges, degree_seq, q)
        H = xgi.Hypergraph(H)

        # compute DB and critical infection rate
        DB, _ = compute_DB_matrix(H)
        _, beta_crit = compute_sir_threshold(DB, mu=gamma)
        beta_crits.append(beta_crit)

        print(f"q={q:.2f} → β_crit={beta_crit:.6f}")

In [ ]:
dataset = datasets[1]
H = xgi.load_xgi_data(dataset)
num_nodes = H.num_nodes
edges = edge_size_distribution(H)
degree_seq = [H.degree(n) for n in H.nodes]
gamma = 0.05
colors = ["#00B388", "#DA291C", "#418FDF"]
num_graphs = 10  # Number of graphs to simulate

# Compute critical infection rates for all datasets first
# compute_Bcrit_for_datasets(mu=gamma)

compute_Bcrit_for_q(num_nodes, edges, degree_seq, gamma, colors, num_graphs)

IndexError: invalid index to scalar variable.